# Grading students' submissions

## Overall solution
The CLI interface for the autograder works only with docker containers.  
If we want to grade without containers, we will need to use other means.

In this notebook, we use the `grade_submission` API.  

**Limitations:** this method is sequential and may take a lot of time if there are lots of submission

## Grading process

* Step 1: grade the individual submissions, possibility to generate 2 CSV files for each student: 1 with the overall grade and 1 with the detailed grading
* Step 2: fill out the individual CSV from moodle for each student
* Step 3: concatenate all moodle CSV files into 1 global gradebook

In [1]:
from otter.api import grade_submission # grading api
import glob # patterns in accessing file paths
import pandas as pd # pandas for saving as csv
pd.set_option('display.max_colwidth', None)
import csv # csv quoting options
from datetime import datetime # date formatting
from IPython.display import display # display for debug
import itertools as it # iteration tools
import os # file management tools

## Grading utility functions

In [2]:
def grade_and_writeCSV(submissionfile, graderzip, graderoutputfolder, 
                       graderdetailsfilename="graderdetails", graderresultfilename="graderresult", 
                       includetestcasemessages=False):
    
    # Call the autograder and get the result
    gradingoutput = grade_submission(submissionfile, graderzip, quiet=True)
    
    # Gather the total grade for the submission and save it
    graderresultdf = pd.DataFrame([[submissionfile, gradingoutput.total, gradingoutput.possible]], 
                                  columns=["Submission Name", "Grade", "Possible"])

    # Gather the details of the grading
    detailedresults = []
    for exercisename, exercise in gradingoutput.results.items():
        testcasemessage = ""
        
        # Iterate over the test results to collect the feedback messages if any
        if includetestcasemessages:
            for test_case_result in exercise.test_case_results:
                if (test_case_result.passed and (test_case_result.test_case.success_message is not None)):
                    testcasemessage+= test_case_result.test_case.name+": "+test_case_result.test_case.success_message+"\n"
                elif ((not test_case_result.passed) and (test_case_result.test_case.failure_message is not None)):
                    testcasemessage+= test_case_result.test_case.name+": "+test_case_result.test_case.failure_message+"\n"
        
        detailedresults.append([exercise.name, exercise.score, exercise.possible, testcasemessage])
    
    graderdetailsdf = pd.DataFrame(detailedresults, columns=["Exercise", "Grade", "Possible", "Feedback"])

    # Saving the results to CSV files
    # Creating the output folder if it does not exist
    if not os.path.exists(graderoutputfolder):
        os.mkdir(graderoutputfolder)

    # Saving the overall grade
    graderresultfile = graderoutputfolder+"/"+graderresultfilename+".csv"
    graderresultdf.to_csv(graderresultfile, index=False, quoting=csv.QUOTE_NONNUMERIC)

    # Saving the details
    graderdetailsfile = graderoutputfolder+"/"+graderdetailsfilename+".csv"
    graderdetailsdf.to_csv(graderdetailsfile, header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)
    
        
    return graderresultdf, graderdetailsdf

In [3]:
def write_moodleCSV(moodlegradebookfile, graderresultdf, graderdetailsdf, 
                    includedetailsgrades=True, includedetailsmsgs=True):

    # Read the moodle file
    moodledf = pd.read_csv(moodlegradebookfile, skip_blank_lines=True)
    #display(moodledf)
    
    if includedetailsgrades:
        # Formatters to get a pretty rendering of the info
        exerciseformatter = lambda x: 'Question %s:' % x
        gradeformatter = lambda x: '%s /' % x
        msgformatter = lambda x: '=> Messages: %s' % x.replace("\n", "")
        
        # Transform the details into string
        details = graderdetailsdf.to_string(index=False, header=False, 
                                         columns=["Exercise", "Grade", "Possible"] + (["Feedback"] if includedetailsmsgs else []),
                                         formatters={'Exercise': exerciseformatter, 'Grade': gradeformatter, 'Feedback': msgformatter})
        
        # Add the details to the moodle info
        moodledf.loc[0, "Feedback comments"] = details

        
    # Modify the moodle info
    moodledf.loc[0, "Grade"] = graderresultdf.loc[0, "Grade"]
    #moodledf.loc[0, "Maximum Grade"] = graderresultdf.loc[0, "Possible"] # Actually this line is useless, it is not used by moodle
    moodledf.loc[0, "Last modified (grade)"] = datetime.today().strftime('%A, %d %B %Y %H:%M') # This line is important for the file to be read by moodle!!!
    
    
    # Write the moodle file
    moodledf.to_csv(moodlegradebookfile, index=False, quoting=csv.QUOTE_NONNUMERIC) # The quotes are important for the file to be read by moodle
    
    return moodledf

## Grading and generation of the output

First retrieving the grader:

In [4]:
# Folder in which the assignment and grader has been generated
distributionfolder = "dist"

# Name of the assignment **file**
assignmentname = "assignment"

# Retrieving the grader zip
graderzip = glob.glob(distributionfolder+"/autograder/"+assignmentname+"-autograder_*.zip")[0]
graderzip

'dist/autograder/assignment-autograder_2022_10_25T17_25_42_055684.zip'

Then retrieving students' submissions

In [5]:
# Folder in which to find students' submission folders
allsubmissionsfolder = "moodlesubmissions/"

# Listing all the submissions (folders)
submissionlist = glob.glob(allsubmissionsfolder+"/*/")
submissionlist

['moodlesubmissions/greatassignment_course15917_student218401_submission582673_Cécile_Hardebolle/']

Iterating over submissions, calling the grader and storing the results into CSV files

In [6]:
%%time

# Folder where to find the moodle grading sheets
moodlegradingsheetfolder = "gradebook"

# List of generated moodle data
moodledata = []

# Iterating over submission folders
for submissionfolder in submissionlist:
    
    # Finding the notebook to grade
    submissionfile = submissionfolder+"/"+assignmentname+".ipynb"
    #print(submissionfile)
    
    # Grading
    graderresultdf, graderdetailsdf = grade_and_writeCSV(submissionfile, graderzip, submissionfolder+"/"+moodlegradingsheetfolder, includetestcasemessages=True)
    #display(graderresultdf)
    #display(graderdetailsdf)
    
    # Updating the moodle CSV file
    moodlegradebookfile = glob.glob(submissionfolder+"/"+moodlegradingsheetfolder+"/*_grading.csv")[0] # TODO here do some error management
    moodledf = write_moodleCSV(moodlegradebookfile, graderresultdf, graderdetailsdf)
    #display(moodledf)
    
    moodledata.append(moodledf)

# Saving all the moodle data to a general moodle CSV grading sheet
moodledatadf = pd.concat(moodledata)
#display(moodledata)
moodledatadf.to_csv(allsubmissionsfolder+"/"+"overall_grading.csv", index=False, quoting=csv.QUOTE_NONNUMERIC) # The quotes are important for the file to be read by moodle

CPU times: user 121 ms, sys: 30.2 ms, total: 151 ms
Wall time: 211 ms
